In [1]:
import urllib.request
import shutil
import csv
import os
import pandas as pd
import tea

In [2]:
# Download and save data locally
states_url = 'https://covidtracking.com/api/states.csv'
states_local_path = 'states.csv'

# https://stackoverflow.com/questions/7243750/download-file-from-web-in-python-3
with urllib.request.urlopen(states_url) as response, open(states_local_path, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

In [3]:
data = pd.read_csv(states_local_path)
print(data.columns)


Index(['state', 'positive', 'positiveScore', 'negativeScore',
       'negativeRegularScore', 'commercialScore', 'grade', 'score', 'negative',
       'pending', 'hospitalized', 'death', 'total', 'lastUpdateEt',
       'checkTimeEt', 'dateModified', 'dateChecked'],
      dtype='object')


In [15]:
# Tea: Load data from URL. Could also load data from local copy.
tea.data(states_local_path)

In [24]:
# Tea: Specify variables of interest in dataset
variables = [
    {
        'name' : 'positive',
        'data type' : 'ratio'   # Options: 'nominal', 'ordinal', 'interval', 'ratio'
    },
    {
        'name' : 'positiveScore',
        'data type' : 'ratio'   # Options: 'nominal', 'ordinal', 'interval', 'ratio'
    },
    {
        'name' : 'pending',
        'data type' : 'ratio'
    },
    {
        'name' : 'death',
        'data type' : 'ratio'
    }
]

tea.define_variables(variables)

In [25]:
# Tea: [OPTIONAL] We don't have any assumptions, so we can skip this step. 
assumptions = {
    'Type I (False Positive) Error Rate': 0.05
}

tea.assume(assumptions)

In [28]:
# Tea: Specify experimental design
experimental_design = {
                        'study type': 'observational study',   # 'study type' could be 'experiment'
                        'contributor variables': ['positive', 'negative'],   # 'experiment's have 'independent variables'
                        'outcome variables': 'death',   # 'experiment's have 'dependent variables'
                    }
tea.define_study_design(experimental_design)

In [ ]:
# Tea: State and test hypothesis
tea.hypothesize(['positive', 'positiveScore'], ['positive ~ positiveScore'])

> /usr/local/lib/python3.7/site-packages/tea/z3_solver/solver.py(159)apply()
-> assert len(test_vars) == len(self.test_vars)
